In [4]:
# ======================================================================
# 2. Importaciones iniciales
# ======================================================================
import pandas as pd
import numpy as np
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset


ImportError: cannot import name 'BaseResult' from 'evidently.core' (c:\Users\taran\miniconda3\envs\base_finalML_venv\lib\site-packages\evidently\core\__init__.py)

In [8]:
import os
folder = r"c:\Users\taran\miniconda3\envs\base_finalML_venv\lib\site-packages"
[x for x in os.listdir(folder) if 'evidently' in x.lower()]


[]

In [9]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset


ImportError: cannot import name 'BaseResult' from 'evidently.core' (c:\Users\taran\miniconda3\envs\base_finalML_venv\lib\site-packages\evidently\core\__init__.py)

In [3]:
# ======================================================================
# 3. Cargar tus datos de entrenamiento
# ======================================================================

# Cambia este nombre por tu archivo real
TRAIN_PATH = "notebooks/heart_preprocessed.csv"

df_train = pd.read_csv(TRAIN_PATH)
df_train.head()


,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,FastingBS_1,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up,HeartDisease_1
0,-1.433140,0.415002,0.858035,1.382928,-0.832432,True,True,False,False,False,True,False,False,False,True,False
1,-0.478484,1.527329,-1.184227,0.754157,0.105664,False,False,True,False,False,True,False,False,True,False,True
2,-1.751359,-0.141161,0.745617,-1.525138,-0.832432,True,True,False,False,False,False,True,False,False,True,False
3,-0.584556,0.303769,-0.547191,-1.132156,0.574711,False,False,False,False,False,True,False,True,True,False,True
4,0.051881,0.971166,-0.903182,-0.581981,-0.832432,True,False,True,False,False,True,False,False,False,True,False


In [ ]:
# ======================================================================
# 4. Definir Reference y Current
#    OPCIÓN A: reference = training, current = training (solo para probar)
# ======================================================================

reference_data = df_train.copy()
current_data = df_train.copy()

print("Reference shape:", reference_data.shape)
print("Current shape:  ", current_data.shape)

# ======================================================================
# 5. (Opcional) Simular datos nuevos para ver drift REAL
# ======================================================================

simulate = True   # Cambiar a False si no quieres simular drift real

if simulate:
    df_simulated = df_train.copy()

    # Identificar columnas numéricas
    numeric_cols = df_simulated.select_dtypes(include=[np.number]).columns

    # Introducir drift artificial
    for col in numeric_cols:
        df_simulated[col] = df_simulated[col] * np.random.uniform(
            0.90, 1.15, size=len(df_simulated)
        )

    # Mezclar filas para simular cambio de distribución
    df_simulated = df_simulated.sample(frac=1, random_state=42).reset_index(drop=True)

    current_data = df_simulated.copy()

print("Data actual lista (simulada con drift).")
current_data.head()

# ======================================================================
# 6. Crear el reporte de Data Drift
# ======================================================================

drift_report = Report(metrics=[DataDriftPreset()])

drift_report.run(
    reference_data=reference_data,
    current_data=current_data
)

print("Reporte generado.")


In [ ]:
# ======================================================================
# 7. Guardar el reporte en HTML
# ======================================================================

OUTPUT = "drift_report.html"
drift_report.save_html(OUTPUT)

print(f"Drift report guardado como: {OUTPUT}")


In [ ]:
# ======================================================================
# 8. (Opcional) Visualizar dentro del notebook
# ======================================================================

from IPython.display import IFrame

IFrame(src="drift_report.html", width="100%", height="600px")
